In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Database Reading and Manipulation
import pandas as pd

# Linear Algebra
import numpy as np

# Plotting
import matplotlib.pyplot as plt

# Time
import time

# Random and os for reproducibility
import random
import os

# Model Selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

# Modeling
import tensorflow as tf

# Processing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

# Metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score

# Pipeline
from sklearn.pipeline import Pipeline

# Data imputation
from sklearn.impute import SimpleImputer

# Making keras compatible with scikit learn api
# https://scikit-learn.org/stable/developers/develop.html
from sklearn.base import BaseEstimator, RegressorMixin

# Custom modules
## Model selection
from src.cross_validation.blocking_time_series_split import BlockingTimeSeriesSplit

## Function to print scores
from src.utils.print_scores import print_scores

## Function to calculate score regression metrics
from src.utils.score_regression_metrics import score_regression_metrics

## Function to fill the results metric dict
from src.utils.fill_results_dict import fill_results_dict

# Converting Times Series Data to 3D format
from src.utils.split_sequences import split_sequences

2023-09-16 16:26:45.473107: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-16 16:26:45.476651: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-16 16:26:45.553642: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-16 16:26:45.555415: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-16 16:26:46.883508: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

<IPython.core.display.Javascript object>

# Functions and definitions

### Data preprocessing

In [3]:
def impute_data(dataset, imputer=None, imputer_params=None):
    x_train = dataset["x_train"]
    x_test = dataset["x_test"]

    # Apply imputation to the data
    if imputer is not None:
        imputer = imputer() if imputer_params is None else imputer(**imputer_params)
        x_train = imputer.fit_transform(x_train)
        x_test = imputer.transform(x_test)

    dataset["x_train"] = x_train
    dataset["x_test"] = x_test

    return dataset

<IPython.core.display.Javascript object>

In [4]:
def transform_data(dataset, transformer=None):
    x_train = dataset["x_train"]
    x_test = dataset["x_test"]

    # Apply data normalization/standardization to the data
    if transformer is not None:
        scaler = transformer()
        x_train = scaler.fit_transform(x_train)
        x_test = scaler.transform(x_test)

    dataset["x_train"] = x_train
    dataset["x_test"] = x_test

    return dataset

<IPython.core.display.Javascript object>

In [5]:
def preprocess_data(dataset, transformer=None, imputer=None, imputer_params=None):
    dataset = impute_data(dataset, imputer, imputer_params)
    dataset = transform_data(dataset, transformer)
    return dataset

<IPython.core.display.Javascript object>

### Train and evaluate the model

In [6]:
def train_and_evaluate_model(Estimator, dataset, estimator_params=None):
    """
    Purpose: Helper function to be used in conjunction with
    blocked time_series cross validation function
    """
    x_train = dataset["x_train"]
    y_train = dataset["y_train"]
    x_test = dataset["x_test"]
    y_test = dataset["y_test"]

    # Instantiate the model
    model = Estimator() if estimator_params is None else Estimator(**estimator_params)

    # Fitting the model
    model.fit(x_train, y_train)

    # Making predictions on train/test sets
    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)

    # Return regression metrics
    return score_regression_metrics(y_train, y_train_pred, y_test, y_test_pred)

<IPython.core.display.Javascript object>

## Sequence helper

In [7]:
def generate_sequences(dataset, timesteps):
    dataset["x_train"], dataset["y_train"] = split_sequences(
        pd.concat(
            [
                pd.DataFrame(dataset["x_train"], columns=x.columns),
                dataset["y_train"],
            ],
            axis=1,
        ).values,
        timesteps,
    )

    dataset["x_test"], dataset["y_test"] = split_sequences(
        pd.concat(
            [
                pd.DataFrame(dataset["x_test"], columns=x.columns),
                dataset["y_test"],
            ],
            axis=1,
        ).values,
        timesteps,
    )
    return dataset

<IPython.core.display.Javascript object>

### Custom Cross Validate

In [8]:
def custom_cross_validate(
    Estimator,
    Imputer,
    Transform,
    x,
    y,
    cv,
    timesteps,
    dates=None,
    estimator_params=None,
    imputer_params=None,
):
    results = []
    scores = []

    for train_index, test_index in cv.split(x):
        dataset = {
            "dates_train": dates[train_index].reset_index(drop=True),
            "x_train": x.loc[train_index].reset_index(drop=True),
            "y_train": y[train_index].reset_index(drop=True),
            "dates_test": dates[test_index].reset_index(drop=True),
            "x_test": x.loc[test_index].reset_index(drop=True),
            "y_test": y[test_index].reset_index(drop=True),
        }

        # Preprocess the dataset
        dataset = preprocess_data(dataset, Transform, Imputer, imputer_params)

        # generate sequences (3D format)
        dataset = generate_sequences(dataset, timesteps)

        # Train and Evaluate the model
        score = train_and_evaluate_model(Estimator, dataset, estimator_params)
        scores.append(score)

    # After every iteration metrics results are appended together
    scores_final = {key: [] for key, _ in scores[0].items()}
    for scores_dict in scores:
        for key, value in scores_dict.items():
            scores_final[key] += [value]
    results.append(scores_final)
    return results

<IPython.core.display.Javascript object>

## Model Definition

In [9]:
class LSTM(BaseEstimator, RegressorMixin):
    def __init__(self):
        self.model = self.get_model()
        self.batch_size = 16
        self.epochs = 300
        self.verbose = 0

    def fit(self, X=None, y=None):
        self.model.fit(
            X, y, batch_size=self.batch_size, epochs=self.epochs, verbose=self.verbose
        )

    def predict(self, X=None):
        return self.model.predict(X, verbose=self.verbose)

    def get_model(self):
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.LSTM(units=16, activation="relu"))
        model.add(tf.keras.layers.Dropout(rate=0.10))
        model.add(tf.keras.layers.Dense(units=1))
        model.compile(
            # optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
            optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001),
            loss="mse",
            metrics=[tf.keras.metrics.RootMeanSquaredError(name="RMSE")],
        )
        return model

<IPython.core.display.Javascript object>

# Settings for Reproducibility

In [10]:
def set_seeds():
    os.environ["CUDA_VISIBLE_DEVICES"] = ""
    os.environ["PYTHONHASHSEED"] = str(SEED)
    tf.random.set_seed(SEED)
    np.random.seed(SEED)
    random.seed(SEED)


# https://stackoverflow.com/questions/36288235/how-to-get-stable-results-with-tensorflow-setting-random-seed

<IPython.core.display.Javascript object>

In [11]:
def set_global_determinism():
    set_seeds(seed=SEED)

    os.environ["TF_DETERMINISTIC_OPS"] = "1"
    os.environ["TF_CUDNN_DETERMINISTIC"] = "1"

    tf.config.threading.set_inter_op_parallelism_threads(1)
    tf.config.threading.set_intra_op_parallelism_threads(1)

<IPython.core.display.Javascript object>

In [12]:
SEED = 47
METRICS = (
    "neg_root_mean_squared_error",
    "neg_mean_absolute_error",
    "neg_mean_absolute_percentage_error",
    "r2",
)
METRICS_DICT = {
    "neg_root_mean_squared_error": "RMSE",
    "neg_mean_absolute_error": "MAE",
    "neg_mean_absolute_percentage_error": "MAPE",
    "r2": "R2",
}

<IPython.core.display.Javascript object>

## Dataframe structure to save the results

In [13]:
results_to_save = []

results_dict = {
    "Category": "Local Model",
    "Company": "partner_i",
    "Features": "Chemical + Mineralogical + Physical",
    "Data Shape": None,
    "Timesteps": None,
    "Model": "LSTM",
    "Model Params": None,
    "Scaler": "Standard Scaler",
    "Scaler Params": None,
    "Imputer": "Median",
    "Imputer Params": None,
    "Cross Validation": None,
    "Cross Validation Params": np.nan,
    "RMSE Train": np.nan,
    "MAE Train": np.nan,
    "MAPE Train": np.nan,
    "R2 Train": np.nan,
    "RMSE Test": np.nan,
    "MAE Test": np.nan,
    "MAPE Test": np.nan,
    "R2 Test": np.nan,
}

<IPython.core.display.Javascript object>

# Reading the dataset

In [14]:
df = pd.read_csv("../../../../../data/processed/partner_i-Oficial/cpiif40.csv")

<IPython.core.display.Javascript object>

## Defining Features

In this set of experiments we use all available features

In [15]:
df_copy = df.copy()

<IPython.core.display.Javascript object>

<h2>1. Dataset: df_copy</h2> <br>In this dataset all features are used.

In [16]:
dates = df["Date"].copy()
y = df_copy.pop("CS28")
x = df_copy
df_copy = pd.concat([x, y], axis=1)

<IPython.core.display.Javascript object>

In [17]:
TIMESTEPS_LIST = [1, 7, 14]

<IPython.core.display.Javascript object>

# 1. Long Short Term Memory - LSTM

## 1.1 Repeated KFold Cross validation

<b>Dataset shape:</b> (594, 38)<br>
<b>Repeats:</b>10<br>
<b>Splits:</b>10<br>
    1. 10 folds of 59 samples each
    2. 90% train (535 samples each fold)
    3. 10% test (59 samples each fold)
<b>Total:</b> 100 models<br>

In [18]:
start = time.time()

repeats = 3
n_splits = 5
TIMESTEPS_LIST = [1, 7, 14]

print("Repeated Cross Validation:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()

for timesteps in TIMESTEPS_LIST:
    set_seeds()
    cv = RepeatedKFold(n_splits=n_splits, n_repeats=repeats, random_state=SEED)
    x = df_copy.drop(["Date", "CS28"], axis=1)
    y = df_copy["CS28"]
    scores = custom_cross_validate(
        LSTM,
        SimpleImputer,
        StandardScaler,
        x,
        y,
        cv,
        timesteps=timesteps,
        dates=dates,
        estimator_params=None,
        imputer_params={"strategy": "median"},
    )
    scores = scores[0]
    print("TIMESTEPS: %d " % timesteps)
    print_scores(scores, METRICS, METRICS_DICT)

    results_dict_copy = results_dict.copy()
    results_dict_copy["Timesteps"] = timesteps
    results_dict_copy["Cross Validation"] = "Repeated KFold"
    results_dict_copy["Cross Validation Params"] = '{"N_Splits": 5, "Repeats": 3}'
    results_dict_copy["Data Shape"] = x.shape
    df_results = fill_results_dict(results_dict_copy, scores)
    results_to_save.append(df_results)

end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Repeated Cross Validation:
Repeats: 3
n_splits: 5



2023-09-16 16:26:49.921463: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-09-16 16:26:49.921511: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: inspirada
2023-09-16 16:26:49.921517: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: inspirada
2023-09-16 16:26:49.921679: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 525.125.6
2023-09-16 16:26:49.921702: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 525.125.6
2023-09-16 16:26:49.921706: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 525.125.6


TIMESTEPS: 1 
******
[TRAIN]
******
RMSE: 2.571 (0.261)
MAE: 1.962 (0.186)
MAPE: 0.041 (0.004)
R2: -3.812 (1.036)


******
[TEST]
******
RMSE: 5.832 (2.611)
MAE: 3.584 (0.519)
MAPE: 0.075 (0.011)
R2: -28.816 (30.252)


TIMESTEPS: 7 
******
[TRAIN]
******
RMSE: 1.302 (0.241)
MAE: 1.053 (0.204)
MAPE: 0.022 (0.004)
R2: -0.274 (0.473)


******
[TEST]
******
RMSE: 4.578 (4.360)
MAE: 2.836 (1.208)
MAPE: 0.060 (0.025)
R2: -28.608 (76.092)


TIMESTEPS: 14 
******
[TRAIN]
******
RMSE: 1.176 (0.263)
MAE: 0.948 (0.233)
MAPE: 0.020 (0.005)
R2: -0.053 (0.513)


******
[TEST]
******
RMSE: 3.401 (2.012)
MAE: 2.423 (0.964)
MAPE: 0.051 (0.020)
R2: -9.816 (15.353)


Minutes Elapsed:  29.34633145729701


<IPython.core.display.Javascript object>

## 1.2. Blocking Time Series Cross Validation

<b>Dataset shape:</b> (594, 38)<br>
<b>Splits:</b>5<br>    
    1. 5 folds of 118 samples
    2. 50% train (59 samples each fold)
    3. 50% test (59 samples each fold)
<b>Total:</b> 5 models<br>

In [19]:
start = time.time()

repeats = 3
n_splits = 5
train_size = 0.8
TIMESTEPS_LIST = [1, 7, 14]

print("Blocking Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()

for timesteps in TIMESTEPS_LIST:
    set_seeds()
    scores_final = None

    for _ in range(repeats):
        x = df_copy.drop(["Date", "CS28"], axis=1)
        y = df_copy["CS28"]

        cv = BlockingTimeSeriesSplit(n_splits=n_splits, train_size=train_size)

        scores = custom_cross_validate(
            LSTM,
            SimpleImputer,
            StandardScaler,
            x,
            y,
            cv,
            timesteps,
            dates=dates,
            estimator_params=None,
            imputer_params={"strategy": "median"},
        )
        scores = scores[0]
        if scores_final is None:
            scores_final = {key: [] for key, _ in scores.items()}

        for key, value in scores.items():
            scores_final[key] += [value]

    print("TIMESTEPS: %d " % timesteps)
    print_scores(scores_final, METRICS, METRICS_DICT)

    # Saving the results
    scores = {key: np.array(val).flatten() for key, val in scores_final.items()}
    results_dict_copy = results_dict.copy()
    results_dict_copy["Timesteps"] = timesteps
    results_dict_copy["Cross Validation"] = "Blocking Time Series Split"
    results_dict_copy[
        "Cross Validation Params"
    ] = '{"N_Splits": 5, "Repeats": 3, "train_size": 0.8}'
    results_dict_copy["Data Shape"] = x.shape
    df_results = fill_results_dict(results_dict_copy, scores)
    results_to_save.append(df_results)

end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Blocking Time Series Split:
Repeats: 3
n_splits: 5

TIMESTEPS: 1 
******
[TRAIN]
******
RMSE: 6.996 (1.248)
MAE: 5.288 (0.830)
MAPE: 0.111 (0.018)
R2: -41.544 (20.595)


******
[TEST]
******
RMSE: 14.168 (2.304)
MAE: 11.662 (2.489)
MAPE: 0.246 (0.053)
R2: -320.073 (148.360)


TIMESTEPS: 7 
******
[TRAIN]
******
RMSE: 1.599 (0.272)
MAE: 1.274 (0.208)
MAPE: 0.027 (0.004)
R2: -1.253 (1.115)


******
[TEST]
******
RMSE: 12.399 (5.365)
MAE: 10.683 (5.098)
MAPE: 0.226 (0.108)
R2: -448.281 (598.989)


TIMESTEPS: 14 
******
[TRAIN]
******
RMSE: 2.327 (0.604)
MAE: 1.904 (0.510)
MAPE: 0.040 (0.011)
R2: -3.799 (2.596)


******
[TEST]
******
RMSE: 13.499 (10.265)
MAE: 11.809 (9.811)
MAPE: 0.250 (0.207)
R2: -950.876 (1892.112)


Minutes Elapsed:  9.1033460577329


<IPython.core.display.Javascript object>

## 1.3. Time Series Split Cross Validation

The training set has size i * n_samples // (n_splits + 1) + n_samples % (n_splits + 1) in the i th split, with a test set of size n_samples//(n_splits + 1) by default, where n_samples is the number of samples.


<b>Dataset shape:</b> (594, 38)<br>
<b>Splits:</b>10<br>    
    1. Train: 10 folds of 54, 108, 162, 216, 270, 324, 378, 432, 486 samples each fold
    2. Test: 54 samples each fold
<b>Total:</b> 10 models<br>

In [20]:
set_seeds()
start = time.time()
gap = 0
n_splits = 5
repeats = 3
TIMESTEPS_LIST = [1, 7, 14]

print("Blocking Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()

for timesteps in TIMESTEPS_LIST:
    set_seeds()
    scores_final = None

    for _ in range(repeats):
        x = df_copy.drop(["Date", "CS28"], axis=1)
        y = df_copy["CS28"]

        cv = TimeSeriesSplit(
            gap=gap, max_train_size=None, n_splits=n_splits, test_size=None
        )
        scores = custom_cross_validate(
            LSTM,
            SimpleImputer,
            StandardScaler,
            x,
            y,
            cv,
            timesteps,
            dates=dates,
            estimator_params=None,
            imputer_params={"strategy": "median"},
        )
        scores = scores[0]
        if scores_final is None:
            scores_final = {key: [] for key, _ in scores.items()}

        for key, value in scores.items():
            scores_final[key] += [value]

    print("TIMESTEPS: %d " % timesteps)
    print_scores(scores_final, METRICS, METRICS_DICT)

    # Saving the results
    scores = {key: np.array(val).flatten() for key, val in scores_final.items()}
    results_dict_copy = results_dict.copy()
    results_dict_copy["Timesteps"] = timesteps
    results_dict_copy["Cross Validation"] = "Time Series Split"
    results_dict_copy[
        "Cross Validation Params"
    ] = '{"N_Splits": 5, "Repeats": 3, "Gap": 0}'
    results_dict_copy["Data Shape"] = x.shape
    df_results = fill_results_dict(results_dict_copy, scores)
    results_to_save.append(df_results)

end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Blocking Time Series Split:
Repeats: 3
n_splits: 5

TIMESTEPS: 1 
******
[TRAIN]
******
RMSE: 4.496 (2.108)
MAE: 3.321 (1.513)
MAPE: 0.070 (0.032)
R2: -18.825 (17.219)


******
[TEST]
******
RMSE: 12.141 (5.952)
MAE: 8.876 (4.931)
MAPE: 0.187 (0.104)
R2: -198.740 (185.817)


TIMESTEPS: 7 
******
[TRAIN]
******
RMSE: 1.472 (0.295)
MAE: 1.211 (0.288)
MAPE: 0.026 (0.006)
R2: -1.008 (0.862)


******
[TEST]
******
RMSE: 10.043 (4.439)
MAE: 7.465 (2.914)
MAPE: 0.157 (0.062)
R2: -131.010 (112.273)


TIMESTEPS: 14 
******
[TRAIN]
******
RMSE: 1.692 (0.562)
MAE: 1.433 (0.543)
MAPE: 0.030 (0.011)
R2: -1.823 (2.091)


******
[TEST]
******
RMSE: 10.307 (7.645)
MAE: 7.983 (5.895)
MAPE: 0.169 (0.125)
R2: -189.685 (255.622)


Minutes Elapsed:  24.430998555819194


<IPython.core.display.Javascript object>

## 1.4. Out of time Split Cross Validation

<b>Dataset shape:</b> (594, 38)<br>
<b>Train size: 80%</b><br>
<b>Test  size: 20%</b>


<b>Splits:</b> 2<br>    
    1. Train: 475
    2. Test: 118
<b>Total:</b> 1 model<br>

In [21]:
start = time.time()
test_size = 0.2
repeats = 3
TIMESTEPS_LIST = [1, 7, 14]

print("Out of time Cross Val:")
print(f"Repeats: {repeats}")
print(f"Train: {80}%", f"Test: {20}%")
print()

for timesteps in TIMESTEPS_LIST:
    set_seeds()
    scores_final = None

    for _ in range(repeats):
        # Data Splitting
        x = df_copy.drop(["Date", "CS28"], axis=1)
        y = df_copy["CS28"]

        x_train, x_test, y_train, y_test = train_test_split(
            x, y, test_size=test_size, random_state=SEED, shuffle=False
        )

        # Preprocessing
        imputer = SimpleImputer(strategy="median")
        scaler = StandardScaler()

        x_train = imputer.fit_transform(x_train)
        x_train = scaler.fit_transform(x_train)
        dates_train = dates[: x_train.shape[0]].reset_index(drop=True)

        x_test = imputer.transform(x_test)
        x_test = scaler.transform(x_test)
        dates_test = dates[x_train.shape[0] :].reset_index(drop=True)

        # Sequence Splitting
        data_train = pd.concat(
            [
                pd.DataFrame(x_train, columns=x.columns),
                y_train.reset_index(drop=True),
            ],
            axis=1,
        ).values
        data_test = pd.concat(
            [
                pd.DataFrame(x_test, columns=x.columns),
                y_test.reset_index(drop=True),
            ],
            axis=1,
        ).values

        x_train, y_train = split_sequences(data_train, timesteps)
        x_test, y_test = split_sequences(data_test, timesteps)

        # Train model and test evalutation
        # Fit model
        pipeline = Pipeline([("estimator", LSTM())])
        pipeline.fit(x_train, y_train)

        # Make predictions
        y_train_pred = pipeline.predict(x_train)
        y_test_pred = pipeline.predict(x_test)

        # evaluate predictions
        scores = score_regression_metrics(y_train, y_train_pred, y_test, y_test_pred)

        if scores_final is None:
            scores_final = {key: [] for key, _ in scores.items()}

        for key, value in scores.items():
            scores_final[key] += [value]

    print("TIMESTEPS: %d " % timesteps)
    print_scores(scores_final, METRICS, METRICS_DICT)

    # Saving the results
    # scores = {key: val[0] for key, val in scores.items()}
    results_dict_copy = results_dict.copy()
    results_dict_copy["Timesteps"] = timesteps
    results_dict_copy["Cross Validation"] = "Out of time Split"
    results_dict_copy["Cross Validation Params"] = '{"Test Size": 0.2}'
    results_dict_copy["Data Shape"] = x.shape
    df_results = fill_results_dict(
        results_dict_copy, {key: value for key, value in scores_final.items()}
    )
    results_to_save.append(df_results)

end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Out of time Cross Val:
Repeats: 3
Train: 80% Test: 20%

TIMESTEPS: 1 
******
[TRAIN]
******
RMSE: 2.641 (0.361)
MAE: 2.007 (0.244)
MAPE: 0.042 (0.005)
R2: -4.389 (1.479)


******
[TEST]
******
RMSE: 11.926 (4.286)
MAE: 5.700 (0.359)
MAPE: 0.120 (0.008)
R2: -95.019 (61.185)


TIMESTEPS: 7 
******
[TRAIN]
******
RMSE: 1.443 (0.163)
MAE: 1.175 (0.150)
MAPE: 0.025 (0.003)
R2: -0.614 (0.366)


******
[TEST]
******
RMSE: 8.364 (1.482)
MAE: 5.995 (0.080)
MAPE: 0.126 (0.002)
R2: -45.524 (16.781)


TIMESTEPS: 14 
******
[TRAIN]
******
RMSE: 0.963 (0.058)
MAE: 0.772 (0.049)
MAPE: 0.016 (0.001)
R2: 0.289 (0.087)


******
[TEST]
******
RMSE: 6.922 (0.509)
MAE: 5.204 (0.832)
MAPE: 0.110 (0.017)
R2: -32.923 (5.054)


Minutes Elapsed:  8.225058019161224


<IPython.core.display.Javascript object>

In [22]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation", "Timesteps"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)


Features Model            Cross Validation  \
                                                                            
0   Chemical + Mineralogical + Physical  LSTM  Blocking Time Series Split   
1   Chemical + Mineralogical + Physical  LSTM  Blocking Time Series Split   
2   Chemical + Mineralogical + Physical  LSTM  Blocking Time Series Split   
3   Chemical + Mineralogical + Physical  LSTM           Out of time Split   
4   Chemical + Mineralogical + Physical  LSTM           Out of time Split   
5   Chemical + Mineralogical + Physical  LSTM           Out of time Split   
6   Chemical + Mineralogical + Physical  LSTM              Repeated KFold   
7   Chemical + Mineralogical + Physical  LSTM              Repeated KFold   
8   Chemical + Mineralogical + Physical  LSTM              Repeated KFold   
9   Chemical + Mineralogical + Physical  LSTM           Time Series Split   
10  Chemical + Mineralogical + Physical  LSTM           Time Series Split   
11  Chemical + Mineralogical + Physical  LSTM           Time Series Split   

   Timesteps  RMSE Test              MAE Test           MAPE Test            \
                   mean        std       mean       std      mean       std   
0          1  14.168045   2.303552  11.661769  2.489027  0.246322  0.052874   
1          7  12.399261   5.365340  10.683296  5.098121  0.226197  0.108334   
2         14  13.498748  10.265054  11.808990  9.811229  0.249932  0.206517   
3          1  11.926499   4.286500   5.700343  0.358930  0.119905  0.007578   
4          7   8.363712   1.481653   5.995285  0.080194  0.125881  0.001533   
5         14   6.922133   0.508867   5.204375  0.832285  0.109713  0.017407   
6          1   5.832097   2.611103   3.583756  0.519007  0.075447  0.010969   
7          7   4.578064   4.360041   2.836126  1.208389  0.059736  0.025464   
8         14   3.400799   2.011799   2.423192  0.963704  0.051062  0.020370   
9          1  12.140968   5.951904   8.875621  4.930877  0.186859  0.104242   
10         7  10.043496   4.438852   7.465069  2.913993  0.157490  0.062111   
11        14  10.306690   7.645369   7.983320  5.894659  0.168670  0.125034   

       R2 Test               
          mean          std  
0  -320.073336   148.360051  
1  -448.280523   598.988587  
2  -950.876354  1892.111922  
3   -95.019201    61.185177  
4   -45.523955    16.781000  
5   -32.923452     5.054481  
6   -28.816056    30.251839  
7   -28.607696    76.092334  
8    -9.816182    15.353148  
9  -198.739538   185.817485  
10 -131.010382   112.273240  
11 -189.684912   255.621793

<IPython.core.display.Javascript object>

# Saving the results Dataframe

## Saving the full dataframe

In [23]:
path = "../../../../../reports/results/local_models/partner_i-oficial/cpiif40/full/"
filename = "lstm_results_full_1.csv"

pd.concat(results_to_save).to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>

## Saving the grouped dataframe

In [24]:
cols_groupby = [
    "Category",
    "Company",
    "Data Shape",
    "Timesteps",
    "Features",
    "Model",
    "Cross Validation",
    "Cross Validation Params",
]

cols_agg = ["RMSE Train", "MAE Train", "MAPE Train", "R2 Train"] + [
    "RMSE Test",
    "MAE Test",
    "MAPE Test",
    "R2 Test",
]

path = "../../../../../reports/results/local_models/partner_i-oficial/cpiif40/grouped/"
filename = "lstm_results_grouped_1.csv"


df_results_to_save = (
    pd.concat(results_to_save)
    .groupby(cols_groupby, dropna=False)[cols_agg]
    .agg(["mean", lambda series: pd.Series(series.std(ddof=0), name="std")])
    .reset_index()
    .rename(columns={"<lambda_0>": "std"})
)

df_results_to_save.to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>